In [1]:
from abstractor.train import get_training_batch
from abstractor.utils import AbstractorModel, AbstractorModelRNN
from abstractor.utils import obtain_initial_hidden_states
from bert.utils import obtain_sentence_embeddings
from bert.utils import obtain_word_embeddings
from data.utils import load_training_dictionaries
from extractor.utils import ExtractorModel
from pytorch_transformers import BertModel
from pytorch_transformers import BertTokenizer
from rouge import Rouge

import numpy as np
import torch

In [ ]:
# data = load_training_dictionaries()
# documents, extraction_labels = get_training_batch(data, batch_size=2)

In [ ]:
# # Load data:
# model = AbstractorModelRNN()
# model_path = "results/models/abstractor.pt"
# model.load_state_dict(torch.load(model_path))

In [ ]:
# source_documents, target_summaries = get_training_batch(data, 2)

# # Obtain embeddings
# source_document_embeddings, source_mask, source_tokens = obtain_word_embeddings(
#     model.bert_model, model.bert_tokenizer, source_documents, static_embeddings=False
# )
# target_summary_embeddings, target_mask, target_tokens = obtain_word_embeddings(
#     model.bert_model, model.bert_tokenizer, target_summaries, static_embeddings=True
# )

In [ ]:
# # Obtain extraction probability for each word in vocabulary
# extraction_probabilities, teacher_forcing = model(
#     source_document_embeddings,
#     target_summary_embeddings,
#     teacher_forcing_pct=0
# )  # (batch_size, n_target_words, vocab_size)

# vals, predicted_idx = torch.topk((extraction_probabilities), k=1, dim=2)

# for x in [model.bert_tokenizer.convert_ids_to_tokens(p) for p in predicted_idx.squeeze().tolist()]:
#     print(f"{x}")
#     print()


# # Obtain extraction probability for each word in vocabulary
# extraction_probabilities, teacher_forcing = model(
#     source_document_embeddings,
#     target_summary_embeddings,
#     teacher_forcing_pct=1
# )  # (batch_size, n_target_words, vocab_size)

# vals, predicted_idx = torch.topk((extraction_probabilities), k=1, dim=2)

# for x in [model.bert_tokenizer.convert_ids_to_tokens(p) for p in predicted_idx.squeeze().tolist()]:
#     print(f"{x}")
#     print()

In [2]:
bert_model = BertModel.from_pretrained('bert-base-uncased')
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [3]:
target_tokens = torch.tensor([[  101, 16950,  9215, 22953, 17854,  4232,  2787,  2000,  2507,  1037,
         14234,  2000,  1037,  7985,  1012,  1037,  2047,  3274,  2565,  3271,
          2014, 13445, 12996, 22291,  2015,  2005,  2416, 14234,  5022,  1012,
           102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0],
        [  101,  1037,  6302,  1997,  1037,  2665,  3536,  5051,  9102,  3909,
          2007,  1037, 29268,  2006,  2049,  2067,  2038,  2908, 13434,  2006,
         10474,  1012,  1996,  3746,  2001,  5941,  2011,  5515,  8088,  3235,
          3393,  1011,  2089,  2379,  2414,  1012,  2009, 13977,  1996, 23325,
         15900,  1001, 29268,  5051,  9102,  1998,  2038, 18379,  3365,  2033,
          7834,  1012,   102]])



# target_tokens = torch.flatten(target_tokens)

target_tokens

tensor([[  101, 16950,  9215, 22953, 17854,  4232,  2787,  2000,  2507,  1037,
         14234,  2000,  1037,  7985,  1012,  1037,  2047,  3274,  2565,  3271,
          2014, 13445, 12996, 22291,  2015,  2005,  2416, 14234,  5022,  1012,
           102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0],
        [  101,  1037,  6302,  1997,  1037,  2665,  3536,  5051,  9102,  3909,
          2007,  1037, 29268,  2006,  2049,  2067,  2038,  2908, 13434,  2006,
         10474,  1012,  1996,  3746,  2001,  5941,  2011,  5515,  8088,  3235,
          3393,  1011,  2089,  2379,  2414,  1012,  2009, 13977,  1996, 23325,
         15900,  1001, 29268,  5051,  9102,  1998,  2038, 18379,  3365,  2033,
          7834,  1012,   102]])

In [5]:
target_tokens = torch.roll(target_tokens, dims=1, shifts=-1)  # shift left
target_tokens[:, -1] = 0
[" ".join(bert_tokenizer.convert_ids_to_tokens(tgt_tokens)) for tgt_tokens in target_tokens.tolist()]

['##lly bro ##uss ##ard decided to give a kidney to a stranger . a new computer program helped her donation spur transplant ##s for six kidney patients . [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]',
 'photo of a green wood ##pe ##cker flying with a weasel on its back has gone viral on twitter . the image was snapped by amateur photographer martin le - may near london . it sparked the hash ##tag # weasel ##pe ##cker and has spawned numerous me ##mes . [SEP] [PAD] [PAD]']